In [147]:
import numpy as np

In [148]:
GAMMA = 0.9

MAX_CARS = 20
MAX_CAR_TRANSFERS = 5   
EPSILON = 0.5
COST = -2

ACTIONS = [-5,-4,-3,-2,-1,0,1,2,3,4,5]

policy = np.zeros(shape=(MAX_CARS + 1, MAX_CARS + 1), dtype=int)
value_function = np.zeros(shape=(MAX_CARS + 1, MAX_CARS + 1))

firstFactory= np.load("firstFactory.npy")
secondFactory = np.load("secondFactory.npy")

rewardsFirst = np.load("rewardsFirst.npy")
rewardsSecond = np.load("rewardsSecond.npy")


In [149]:
def update_value_function():
    while True:
        delta = 0
        for carFirst in range(MAX_CARS + 1):
            for carSecond in range(MAX_CARS + 1):
                v = value_function[carFirst, carSecond]
                action = policy[carFirst, carSecond]
                
                finalFirst = carFirst - action
                finalSecond = carSecond + action

                reward = COST*np.abs(action)

                new_value = 0
                for possibleFinalFirst in range(MAX_CARS + 1):
                    for possibleFinalSecond in range(MAX_CARS + 1):
                        new_value += firstFactory[finalFirst, possibleFinalFirst]*secondFactory[finalSecond, possibleFinalSecond]*(reward + rewardsFirst[finalFirst, possibleFinalFirst] + rewardsSecond[finalSecond, possibleFinalSecond] + GAMMA*value_function[possibleFinalFirst, possibleFinalSecond])
                
                delta = max(delta, np.abs(v - new_value))
                value_function[carFirst, carSecond] = new_value
        
        if delta < EPSILON:
            break

def update_policy():
    correct = True
    for carFirst in range(MAX_CARS + 1):
        for carSecond in range(MAX_CARS + 1):
            max_action = 0
            max_value = -10000
            for action in ACTIONS:
                if ((action >= 0 and carFirst - action >= 0 and carSecond + action <= 20) or (action < 0 and carFirst - action <= 20 and carSecond + action >= 0)):
                    finalFirst = carFirst - action
                    finalSecond = carSecond + action
                    
                    reward = COST*np.abs(action)
                    new_value = 0
                    for possibleFinalFirst in range(MAX_CARS + 1):
                        for possibleFinalSecond in range(MAX_CARS + 1):
                            new_value += firstFactory[finalFirst, possibleFinalFirst]*secondFactory[finalSecond, possibleFinalSecond]*(reward + rewardsFirst[finalFirst, possibleFinalFirst] + rewardsSecond[finalSecond, possibleFinalSecond] + GAMMA*value_function[possibleFinalFirst, possibleFinalSecond])
                    
                    if(new_value > max_value):
                        max_value = new_value
                        max_action = action
            
            if not (policy[carFirst, carSecond] == max_action):
                correct = False
            policy[carFirst, carSecond] = max_action
    return correct
            


In [150]:
while True:
    update_value_function()
    if(update_policy()):
        break

np.save("value_function", value_function)
np.save("policy", policy)